In [40]:
import requests
import re
from xml.dom import minidom
from io import StringIO
from bs4 import BeautifulSoup
import numpy as np

In [47]:
def convert_string_to_money(string):
    if string[-1] =='M':
        return float(string[:-1])*10e6
    elif string[-1]=='B':
        return float(string[:-1])*10e9
    else:
        print ('Do not know how to convert ',string)

## info in https://www.crunchbase.com/

## info in www.owler.com

In [6]:
sess = requests.Session()
cookie_string = '''vid=rBEAAlx3/TavdgBFD0weAg==; _ga=GA1.2.1920520798.1551367486; _gid=GA1.2.1286598662.1551367486; D_IID=17663BA9-2CC9-3338-BA20-99E3E53D3C37; D_UID=06420E6A-F63F-3453-8586-536DCB76C943; D_ZID=4FF367A3-37AC-3134-8CF0-EAA3833B3154; D_ZUID=01BA398E-21FF-30B3-AFBC-669A6E8EC229; D_HID=A5A78D32-5811-3456-B812-454855F7143F; D_SID=166.111.156.12:Xf4UWCgWFOXsZESbvf8o+T2ckgiclUsgQNu24uDk0aI; __qca=P0-1364539395-1551367487292; C_EXP=f; NC_VARNISH=true; fts=sNkSEdxpNsq0jvM_UIaWkA; OWLER_WEBID=5c781dfcb3465316f212ae18; cpviewed=111436%2C111436%2C111436; _litra_ses.9b9e=*; OWLER_PC=sb7sYWNyPeOR6Jr7IJZPO0wEJ_f3tJVFa1YKtnyv5xfxLcoGF7O7BaX7-1B-5N2nX1C8k-W4fbExTyuogUWd-SmebHKpO-AQB_YZQcUCVTqtUeLrow_ZrSAAS4s6lwuzeuF79OLQ2oKbEYS1EO9eZw; _litra_id.9b9e=a-016x--b621c0ed-2f28-40fc-987d-1dcf8cdbce78.1551367479.2.1551432675.1551375878.f8859fe9-af39-40de-b5d7-aaa86a430a78; amplitude_id_e102edba5e9caea6b89e3c04fac87a4dowler.com=eyJkZXZpY2VJZCI6IjNkOGU1MGU2LThhODYtNDRkNy04MmFmLTgzMDRlZGE3MzI2ZlIiLCJ1c2VySWQiOiI1NjIwMzc1Iiwib3B0T3V0IjpmYWxzZSwic2Vzc2lvbklkIjoxNTUxNDMyNjM4ODk5LCJsYXN0RXZlbnRUaW1lIjoxNTUxNDMyNjc3NTAzLCJldmVudElkIjozNCwiaWRlbnRpZnlJZCI6MjMsInNlcXVlbmNlTnVtYmVyIjo1N30=; li_c='''
cookies = {(s.split('=')[0]):(s.split('=')[1]) for s in cookie_string.split('; ')}

In [7]:
r = sess.get(url='https://www.owler.com/company/5000fish',
         cookies=cookies, 
         headers={
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
})
soup = BeautifulSoup(r.text, 'html.parser')

##### annual revenue

In [54]:
annual_revenue_string = soup.find_all('p', attrs={'class': 'block stat-value styled-text black'})[0].text
if annual_revenue_string[:4]=='$ < ':
    annual_revenue_float = convert_string_to_money(annual_revenue_string[4:])
else:
    annual_revenue_float = convert_string_to_money(annual_revenue_string)
annual_revenue_float/10e6

1.0

##### competitor count

In [50]:
#print(soup.find_all('span', attrs={'class': 'rank styled-text'}))
len(soup.find_all('span', attrs={'class': 'rank styled-text'}))

10

#####  competitors' revenue

In [51]:
tmp_revenue_list = soup.find_all('div', attrs={'class': 'table-col revenue'})[1:]
competitors_revenue = []
for x in tmp_revenue_list:
    competitors_revenue.append(re.findall(".*>(.*)<.*",str(x))[0][1:])

In [52]:
competitors_revenue_float=[]
for i in competitors_revenue[1:]: #skip the searched company
    if i[:6] ==' &lt; ':
        competitors_revenue_float.append(convert_string_to_money(i[6:]))
    else:
        competitors_revenue_float.append(convert_string_to_money(i))

In [57]:
np.array(competitors_revenue_float).sum()/10e6

2243.8

##### HHI

In [43]:
(np.array(competitors_revenue_float)**2).sum()/((np.array(competitors_revenue_float).sum())**2)

0.3827828875830853

## problems to solve  
- [x]convert string to float
- [ ]company and url mismatch
- [ ]missing value: if a competitor company does not have revenue data, decrease one competitor num